In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
!pip install pytorch_lightning
#!pip install --upgrade pytorch_lightning==0.7.6
import torch.utils as torchU
import pytorch_lightning as torch_light
from torchvision import datasets, models, transforms
import numpy as np
import os
from matplotlib import pyplot as plt
import math

In [ ]:
A = torch.rand((5,3))
print(A)
A = A*2-1
print(A)
A = (A+1)/2
print(A)


In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/Cuda Vision lab assignment 8"
!ls

/content/drive/My Drive/Colab Notebooks/Cuda Vision lab assignment 8
assignment8_V1.ipynb  data  hparams.yaml  lightning_logs


In [ ]:
def build_network(hparams):
  net = []
  for ind in range(len(hparams["layer_type"])):
    if hparams["layer_type"][ind] == "maxpool":
      net.append(nn.MaxPool2d(hparams["layers"][ind][0],hparams["layers"][ind][1],hparams["layers"][ind][2]))
      continue
    if hparams["layer_type"][ind] == "avgpool":
      net.append(nn.AvgPool2d(hparams["layers"][ind][0],hparams["layers"][ind][1],hparams["layers"][ind][2]))
      continue
    if hparams["layer_type"][ind] == "cnn":
      net.append(nn.Conv2d(in_channels=hparams["layers"][ind][0],out_channels=hparams["layers"][ind][1],
                                  kernel_size=hparams["layers"][ind][2],stride=hparams["layers"][ind][3],padding=hparams["layers"][ind][4]))
      continue
    if hparams["layer_type"][ind] == "cnnt":
      net.append(nn.ConvTranspose2d(in_channels=hparams["layers"][ind][0],out_channels=hparams["layers"][ind][1],
                                  kernel_size=hparams["layers"][ind][2],stride=hparams["layers"][ind][3],padding=hparams["layers"][ind][4],
                                  output_padding=hparams["layers"][ind][5]))                             
      continue
    if hparams["layer_type"][ind] == "tanh":
      net.append(nn.Tanh())
      continue
    if hparams["layer_type"][ind] == "sigmoid":
      net.append(nn.Sigmoid())
      continue
    if hparams["layer_type"][ind] == "linear":
      net.append(nn.Linear(hparams["layers"][ind][0],hparams["layers"][ind][1]))
      continue
    if hparams["layer_type"][ind] == "relu":
      net.append(nn.ReLU())
      continue
    if hparams["layer_type"][ind] == "Lrelu":
      net.append(nn.LeakyReLU())
      continue
    if hparams["layer_type"][ind] == "dropout":
      net.append(nn.Dropout(hparams["layers"][ind][0]))
      continue
    if hparams["layer_type"][ind] == "Bnorm":
      net.append(nn.BatchNorm2d(hparams["layers"][ind][0]))
      continue
  return nn.Sequential(*net)

In [ ]:
"""
  Generator class
  structure of hparams(dic):
    latent_dim: dimension of sample space
    layers: array containing the specifications for each layer w.r.t. layer type
    layers_type: array containing which type of layers are stacked
"""
class Generator(nn.Module):
    def __init__(self, hparams):
        super(Generator, self).__init__()
        self.hparams = hparams
        self.net = build_network(self.hparams)
        
    def forward(self, z):
        return self.net(z.view(z.shape[0],z.shape[1],1,1))

In [ ]:
"""
  Discriminator class
  structure of hparams(dic):
    latent_dim: dimension of sample space
    layers: array containing the specifications for each layer w.r.t. layer type
    layers_type: array containing which type of layers are stacked
"""
class Discriminator(nn.Module):
    def __init__(self, hparams):
      super(Discriminator, self).__init__()
      self.hparams = hparams
      self.net = build_network(self.hparams)
      self.d2 = len(self.net)-self.hparams["1d"][0]
    
    def forward(self, x):
      x = self.net[:self.d2](x)
      return self.net[self.d2:](x.view(x.shape[0],-1))

In [ ]:
"""
  Loading the data_set
"""
data_dir = "/content/drive/My Drive/Colab Notebooks/Cuda Vision lab assignment 8/data"
input_dim = 64
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(input_dim),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #250 dataset
        #transforms.Normalize([0.6155, 0.6182, 0.5580],[0.3410, 0.3323, 0.3663])
        #new robot dataset
        #transforms.Normalize([0.4741, 0.4842, 0.4070],[0.2814, 0.2729, 0.2987])
        transforms.Normalize([0.5]*3,[0.5]*3)
        ])
robot_data_set = datasets.ImageFolder(data_dir,data_transforms)

In [ ]:
loader = torchU.data.DataLoader(robot_data_set,batch_size=len(robot_data_set))
for batch in loader:
  batch, labels = batch
  print(batch.max(),batch.min())

tensor(1.) tensor(-1.)


In [ ]:
def compute_means_and_stds(dset):
  loader = torchU.data.DataLoader(dset,batch_size=len(dset),shuffle=False, drop_last=True)

  for batch in loader:
    batch,labels = batch
    batch = batch.float().cuda()
    means = torch.mean(batch,dim=(0,2,3))
    stds = torch.std(batch,dim=(0,2,3))
  return means, stds

In [ ]:
# call only once to compute means and std for data sets
means,stds = compute_means_and_stds(robot_data_set)
print(means,stds)

tensor([-0.0508, -0.0264, -0.1823], device='cuda:0') tensor([0.5643, 0.5441, 0.5994], device='cuda:0')


In [ ]:
def imshow(inp,title=None,normalize = False):
    inp = torchvision.utils.make_grid(inp)
    """Imshow for Tensor."""
    plt.figure(figsize = (30,20))
    if normalize:
      inp = (inp +1)/2
    inp = inp.permute((1, 2, 0))
    plt.imshow(inp.detach().cpu())
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
# Get a batch of training data
loader = torchU.data.DataLoader(robot_data_set,batch_size=32,shuffle=True)
inputs, classes = next(iter(loader))
class_names = robot_data_set.classes

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out,title=[class_names[x] for x in classes],normalize=True)

In [ ]:
"""
  pytorch lightning class for GAN
"""


class GAN(torch_light.LightningModule):

    def __init__(self, hparams):
      super(GAN, self).__init__()
      self.hparams = hparams

      # networks
      self.generator = Generator(self.hparams["gen_hparams"])
      self.discriminator = Discriminator(self.hparams["dis_hparams"])

      # cache for generated images
      self.generated_imgs = None
      self.last_imgs = None
      self.img_list = []

    def forward(self, z):
      return self.generator(z)

    def training_step(self, batch, batch_nb, optimizer_idx):
      imgs, _ = batch
      self.last_imgs = imgs.cuda()

      # train generator
      if optimizer_idx == 0:
          # sample noise
          z = torch.randn(imgs.shape[0], (self.hparams["gen_hparams"])["latent_dim"]).cuda()

          # generate images
          self.generated_imgs = self(z)

          # ground truth result (ie: all fake)
          # put on GPU because we created this tensor inside training_loop
          valid = torch.ones(imgs.size(0), 1).cuda()

          # adversarial loss is binary cross-entropy
          loss_func = self.get_loss_func()
          pred = self.discriminator(self.generated_imgs)
          g_loss = loss_func(pred, valid)
          self.g_loss = g_loss
          self.mean_pred = pred.mean()
          return {"loss":g_loss}

      # train discriminator
      if optimizer_idx == 1:
          
          # Measure discriminator's ability to classify real from generated samples

          # how well can it label as real?
          valid = torch.ones(imgs.size(0), 1).cuda()

          loss_func = self.get_loss_func()
          pred = self.discriminator(imgs)
          d_loss = loss_func(pred, valid)

          # how well can it label as fake?
          fake = torch.zeros(imgs.size(0), 1).cuda()

          fake_loss = loss_func(
              self.discriminator(self.generated_imgs.detach()), fake)

          # discriminator loss is the average of these
          final_d_loss = d_loss + fake_loss
          return {"loss/dis": d_loss,"loss":final_d_loss,"loss/gen" : self.g_loss,"acc/gen": self.mean_pred,"acc/dis": pred.mean(),"loss/dis_fake": fake_loss}
    def training_epoch_end(self, outputs):
      if self.current_epoch%self.hparams["images_saving_per_epoch"]==0:
            self.img_list.append(self.generated_imgs)

      avg_loss_g = torch.stack([x['loss/gen'] for x in outputs]).mean()
      avg_loss_d = torch.stack([x['loss/dis'] for x in outputs]).mean()
      avg_acc_g = torch.stack([x['acc/gen'] for x in outputs]).mean()
      avg_acc_d = torch.stack([x['acc/dis'] for x in outputs]).mean()
      avg_loss_d_fake = torch.stack([x['loss/dis_fake'] for x in outputs]).mean()
      #avg_acc = torch.stack([x['train_acc'] for x in outputs]).mean()
      #logs = {'loss/train': avg_loss,'accuracy/train':avg_acc,'step':self.current_epoch}
      logs = {'loss/train_gen': avg_loss_g,'loss/train_dis': avg_loss_d,
              'step':self.current_epoch,"acc/gen":avg_acc_g,"acc/dis": avg_acc_d,"loss/train_dis_fake": avg_loss_d_fake}
      return {'loss_gen_train': avg_loss_g,"loss_dis_train":avg_loss_d,'log': logs}
    def validation_step(self, batch, batch_idx):
      imgs, _ = batch
      # sample noise
      z = torch.randn(imgs.shape[0], (self.hparams["gen_hparams"])["latent_dim"]).cuda()

      # generate images
      generated_imgs = self(z)

      # ground truth result (ie: all fake)
      # put on GPU because we created this tensor inside training_loop
      valid = torch.ones(imgs.size(0), 1).cuda()

      # adversarial loss is binary cross-entropy
      loss_func = self.get_loss_func()
      pred = self.discriminator(generated_imgs)
      g_loss = loss_func(pred, valid)

      # Measure discriminator's ability to classify real from generated samples

      # how well can it label as real?
      valid = torch.ones(imgs.size(0), 1).cuda()

      loss_func = self.get_loss_func()
      pred_imgs = self.discriminator(imgs)
      d_loss = loss_func(pred_imgs, valid)

      # how well can it label as fake?
      fake = torch.zeros(imgs.size(0), 1).cuda()

      fake_loss = loss_func(
          self.discriminator(generated_imgs), fake
          )

      # discriminator loss is the average of these
      final_d_loss = d_loss + fake_loss
      return {"loss_val/dis": d_loss,"loss_val/dis_fake": fake_loss,"loss_val/gen" : g_loss,"acc_val/gen":pred.mean(),"acc_val/dis": pred_imgs.mean()}
    def validation_epoch_end(self, outputs):
      avg_loss_g = torch.stack([x['loss_val/gen'] for x in outputs]).mean()
      avg_loss_d = torch.stack([x['loss_val/dis'] for x in outputs]).mean()
      avg_acc_g = torch.stack([x['acc_val/gen'] for x in outputs]).mean()
      avg_acc_d = torch.stack([x['acc_val/dis'] for x in outputs]).mean()
      avg_loss_d_fake = torch.stack([x['loss_val/dis_fake'] for x in outputs]).mean()
      logs = {'loss_val/gen': avg_loss_g,'loss_val/dis': avg_loss_d,
              'step':self.current_epoch,"acc_val/gen":avg_acc_g,"acc_val/dis":avg_acc_d,"loss_val/dis_fake": avg_loss_d_fake}
      return {'avg_val_loss_gen': avg_loss_g,'avg_val_loss_dis': avg_loss_d, 'log': logs}
    def test_step(self, batch, batch_idx):
      pass
    def test_epoch_end(self, outputs):
      pass
    def prepare_data(self):
      dataset = robot_data_set
      #len of dataset is 185
      self.trainset, self.valset = torchU.data.random_split(dataset,[150,35]) 
    def train_dataloader(self):
      return torchU.data.DataLoader(self.trainset,self.hparams["batch_size"],shuffle=True,num_workers=4)
    def val_dataloader(self):
      return torchU.data.DataLoader(self.valset,self.hparams["batch_size"],shuffle=False,num_workers=4)
    def configure_optimizers(self):
        lr_g = self.hparams["learning_rates"][0]
        lr_d = self.hparams["learning_rates"][1]

        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr_g,weight_decay=1e-5)
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr_d,weight_decay=1e-4)
        lr_scheduler_g = torch.optim.lr_scheduler.StepLR(opt_g,1,0.999)
        lr_scheduler_d = torch.optim.lr_scheduler.StepLR(opt_d,1,0.999)
        return [opt_g, opt_d], [lr_scheduler_g,lr_scheduler_d]
    def get_loss_func(self):
      if self.hparams["loss_func"]== "crossentropy":
        return nn.CrossEntropyLoss()
      if self.hparams["loss_func"]== "mse":
        return nn.MSELoss()
      if self.hparams["loss_func"]== "kl-div":
        return nn.KLDivLoss()
      if self.hparams["loss_func"]== "bceloss":
        return nn.BCELoss()

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Cuda Vision lab assignment 8 
!ls

/content/drive/My Drive/Colab Notebooks/Cuda Vision lab assignment 8
assignment8_V1.ipynb  data  hparams.yaml  lightning_logs


# specifications for ConvTranspose2d
 torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
                          stride=1, padding=0, output_padding=0,
                          groups=1, bias=True)<br>
 output_height = (height-1) * stride + kernel_size - 2*padding + output_padding
 # specifications for Conv2d
 torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                 stride=1, padding=0, dilation=1,
                 groups=1, bias=True)<br>
output_height = (input_height+2*padding)-kernel_size+stride)/stride       

In [ ]:
"""
  test code for generator
"""

d=128
deconv1 = nn.ConvTranspose2d(100, d*16, 4, 1, 0)
conv = nn.Conv2d(d*8,d*8,3,1,1)
deconv2 = nn.ConvTranspose2d(d*16, d*12, 4, 2, 1)
deconv3 = nn.ConvTranspose2d(d*12, d*8, 4, 2, 1)
deconv4 = nn.ConvTranspose2d(d*8, d*6, 4, 2, 1)
deconv5 = nn.ConvTranspose2d(d*6, d*4, 4, 2, 1)
deconv6 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
deconv7 = nn.ConvTranspose2d(d*2, 3, 4, 2, 1)

keep = nn.ConvTranspose2d(d*12,d*12,5,1,2)


mini_batch = 25
z = torch.randn((mini_batch, 100)).view(-1, 100, 1, 1)
print("1.st ",z.shape)
z = deconv1(z)
print("2.st ",z.shape)
z = deconv2(z)
print("3.st ",z.shape)
z=keep(z)
print("keep shape: ",z.shape)
z = deconv3(z)
print("4.st ",z.shape)
z = conv(z)
print("same shape? ",z.shape)
z = deconv4(z)
print("5.st ",z.shape)
z = deconv5(z)
print("6.st ",z.shape)
z = deconv6(z)
print("7.st ",z.shape)
z = deconv7(z)
print("8.st ",z.shape)

1.st  torch.Size([25, 100, 1, 1])
2.st  torch.Size([25, 2048, 4, 4])
3.st  torch.Size([25, 1536, 8, 8])
keep shape:  torch.Size([25, 1536, 8, 8])
4.st  torch.Size([25, 1024, 16, 16])
same shape?  torch.Size([25, 1024, 16, 16])
5.st  torch.Size([25, 768, 32, 32])
6.st  torch.Size([25, 512, 64, 64])
7.st  torch.Size([25, 256, 128, 128])
8.st  torch.Size([25, 3, 256, 256])


In [ ]:
"""
  hparams for generator
"""
latent_dim = 100
d=64
layers_g = [[latent_dim,d*16,4,1,0,0],[],[d*16],
            [d*16,d*16,3,1,1],[],[d*16],

            [d*16,d*12,4,2,1,0],[],[d*12],
            [d*12,d*12,3,1,1],[],[d*12],

            [d*12,d*8,4,2,1,0],[],[d*8],
            [d*8,d*8,3,1,1],[],[d*8],

            [d*8,d*4,4,2,1,0],[],[d*4],
            [d*4,d*4,3,1,1],[],[d*4],
            
            [d*4,3,4,2,1,0],[],
          ]
layers_type_g = ["cnnt","Lrelu","Bnorm",
                 "cnn","Lrelu","Bnorm",
                 "cnnt","Lrelu","Bnorm",
                 "cnn","Lrelu","Bnorm",
                 "cnnt","Lrelu","Bnorm",
                 "cnn","Lrelu","Bnorm",
                 "cnnt","Lrelu","Bnorm",
                 "cnn","Lrelu","Bnorm",
                 "cnnt","tanh"
                 ]

hparams_g = {}
hparams_g["latent_dim"] = latent_dim
hparams_g["layers"] = layers_g
hparams_g["layer_type"] = layers_type_g

In [ ]:
"""
  test code for discriminator
"""

d=32
conv1 = nn.Conv2d(3, d, 4, 2, 1)
conv2 = nn.Conv2d(d, d*2, 4, 2, 1)
conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1)
conv5 = nn.Conv2d(d*8, d*10, 4, 2, 1)
# conv6 = nn.Conv2d(32, 64, 4, 2, 1)
# conv7 = nn.Conv2d(64, 64, 4, 2, 1)
# conv8 = nn.Conv2d(64, 100, 4, 2, 1)

mini_batch = 16
z = torch.randn((mini_batch,3,64,64))
print("1.st ",z.shape)
z = conv1(z)
print("2.st ",z.shape)
z = conv2(z)
print("3.st ",z.shape)
z = conv3(z)
print("4.st ",z.shape)
z = conv4(z)
print("5.st ",z.shape)
z = conv5(z)
print("6.st ",z.shape)
# z = conv6(z)
# print("7.st ",z.shape)
# z = conv7(z)
# print("8.st ",z.shape)
# z = conv8(z)
# print("9.st ",z.shape)

1.st  torch.Size([16, 3, 64, 64])
2.st  torch.Size([16, 32, 32, 32])
3.st  torch.Size([16, 64, 16, 16])
4.st  torch.Size([16, 128, 8, 8])
5.st  torch.Size([16, 256, 4, 4])
6.st  torch.Size([16, 320, 2, 2])


In [ ]:
"""
  hparams for discriminator
"""
d=32
layers_d = [
            [3,d,4,2,1],[],[d],    
            [d,d*2,4,2,1],[],[d*2], 
            [d*2,d*4,4,2,1],[],[d*4], 
            [d*4,d*8,4,2,1],[],[d*8],
            [d*8,d*10,4,2,1],[],[d*10],
            [d*10*2*2,2048],[],
            [2048,1],[]   
          ]
layers_type_d = ["cnn","Lrelu","Bnorm",
                 "cnn","Lrelu","Bnorm",
                 "cnn","Lrelu","Bnorm",
                 "cnn","Lrelu","Bnorm",
                 "cnn","Lrelu","Bnorm",
                 "linear","sigmoid",
                 "linear","sigmoid"
                 ]

hparams_d = {}
hparams_d["layers"] = layers_d
hparams_d["layer_type"] = layers_type_d
hparams_d["1d"] = [4,2*2*d*10]

In [ ]:
"""
 model details:
  optimizer are: adam
  loss functions given extra as parameter
  layer types are:  sigmoid: [input_dim,output_dim]
                    linear: [input_dim,output_dim]
                    tanh: [input_dim,output_dim]
                    relu: [input_dim,output_dim]
                    dropout: [dropout_value]
                    cnn: [input_channel,output_channel,kernel_size,stride,padding]
                    cnnt: [input_channel,output_channel,kernel_size,stride,padding,output_padding]
                    max/avgpool: [kernel_size,stride,padding]
  layers is a list of list containing the parameters per each layer
"""
epoches=3000
torch_light.seed_everything()

"""
  hparams
"""
hparams={}
hparams["optimizer"]="adam"
hparams["learning_rates"]=[0.00001,0.0000005]
hparams["batch_size"]=pow(2,5)
hparams["gen_hparams"] = hparams_g
hparams["dis_hparams"] = hparams_d
hparams["loss_func"]="bceloss"
hparams["images_saving_per_epoch"]=50
"""
  training and testing
"""
net_GAN = GAN(hparams)
trainer= torch_light.Trainer(max_epochs=epoches,gpus=-1,fast_dev_run=False,checkpoint_callback=False)


No correct seed found, seed set to 741594605
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [ ]:
"""
  Run training on GAN
"""
trainer.fit(net_GAN)

In [ ]:
"""
  Save model as checkpoint
"""

trainer.save_checkpoint("epoches500_lr0.003_0.001_latd512")

In [ ]:
"""
  Load model as checkpoint
"""

netto = VAE.load_from_checkpoint("model_1_MSE_9l_8l.ptl")

In [ ]:
"""
  Test the quality of generator images
"""
#z_vec = torch.empty(size=((25,256,8,8))).normal_(mean=mean,std=std)
z_vec = torch.randn((25,net_GAN.generator.hparams["latent_dim"])).cuda()
norm = nn.BatchNorm2d(net_GAN.generator.hparams["latent_dim"]).cuda()
#z_vec = norm(z_vec)
gen = net_GAN.generator.cuda()
images = gen(z_vec)
imshow(images,normalize=True)

In [ ]:
"""
  Test how generated images changed every x epoches
"""
counter = 1
for imgs in net_GAN.img_list:
  print("epoch:",counter*hparams["images_saving_per_epoch"])
  counter+=1
  imshow(imgs,normalize=True)
  

In [ ]:
%reload_ext tensorboard
%cd /content/drive/My Drive/Colab Notebooks/Cuda Vision lab assignment 8 
!ls

/content/drive/My Drive/Colab Notebooks/Cuda Vision lab assignment 8
assignment8_V1.ipynb  data  hparams.yaml  lightning_logs


In [ ]:
%tensorboard --logdir lightning_logs